# Project 71: Automatic Caption Generation of Images

## Authors: Oihane Cantero and Julen Etxaniz

## Supervisors: Oier Lopez de Lacalle & Eneko Agirre

## Subject: Machine Learning and Neural Networks

## Date: 

## Objectives: 
### 1. Implement from scratch a caption generation model that uses a CNN to condition a LSTM based language model.
### 2. Extend the basic caption generation system that incorporates an attention mechanism to the model.

## Contents: